In [ ]:
!pip install pandas pyarrow

In [ ]:
!git clone https://github.com/nianlonggu/MemSum.git

In [ ]:
!pip install -q -r MemSum/requirements.txt

In [ ]:
import pandas as pd
import json

In [ ]:
df_full_text=pd.read_parquet("../../Dares_1Kaccords_dataset_txt_label.parquet")
df_full_text=df_full_text.drop(columns="label_txt")

In [ ]:
df_summary_label=pd.read_parquet("../../Dares_accords_docx_sommaire_label_230811.parquet")

In [ ]:
df_join_ftext_sum=df_full_text.join(df_summary_label)

In [ ]:
def trim_new_line_on_start(string):
    if string.startswith("\n") or string.startswith(" ")or string.startswith("\t"):
        return trim_new_line_on_start(string[1:])
    else:
        return string

In [ ]:
df_join_ftext_sum["txt_trim"]=df_join_ftext_sum["txt"].apply(trim_new_line_on_start)

In [ ]:
df_join_ftext_sum["txt_trim_list"]=df_join_ftext_sum[["txt_trim"]].apply(lambda x : x.str.split("\n"))

In [ ]:
df_join_ftext_sum["sommaire_label_list"]=df_join_ftext_sum[["sommaire_label"]].apply(lambda x : x.str.split("\n"))

In [ ]:
def return_dict(text,label):
    d=dict()
    d["text"]=text
    d["summary"]=label
    return json.dumps(d)

In [ ]:
df_join_ftext_sum["dict_txt_sommaire"]=df_join_ftext_sum[["txt_trim_list","sommaire_label_list"]].apply(lambda x :return_dict(*x),axis=1)


In [ ]:
df_join_ftext_sum=df_join_ftext_sum.reset_index()[['dict_txt_sommaire']]

In [ ]:
df_1 = df_join_ftext_sum.sample(frac=0.7)  
df_2 = df_join_ftext_sum.loc[~df_join_ftext_sum.index.isin(df_1.index)].reset_index(drop=True)
df_3 = df_2.sample(frac=0.5)  
df_validation = df_2.loc[~df_2.index.isin(df_3.index)].reset_index(drop=True)
df_train = df_1.reset_index(drop=True)
df_test = df_3.reset_index(drop=True)

In [ ]:
df_train=df_train.set_index("dict_txt_sommaire")
df_test=df_test.set_index("dict_txt_sommaire")
df_validation=df_validation.set_index("dict_txt_sommaire")

In [ ]:
df_train.to_csv("train_data.json")
df_validation.to_csv("validation_data.json")
df_test.to_csv("test_data.json")

In [ ]:
!cat train_data.json | tail -n +2 | sed -e 's/"{/{/g' | sed -e 's/}"/}/g'  | sed -e 's/""/"/g' > train_data_parsed.json
!cat validation_data.json | tail -n +2 | sed -e 's/"{/{/g' | sed -e 's/}"/}/g'  | sed -e 's/""/"/g' > validation_data_parsed.json
!cat test_data.json | tail -n +2 | sed -e 's/"{/{/g' | sed -e 's/}"/}/g'  | sed -e 's/""/"/g' > test_data_parsed.json

In [ ]:
!cd MemSum/data_preprocessing; python get_high_rouge_episodes_mp.py -input_corpus_file_name ../../train_data_parsed.json -output_corpus_file_name ../../custom_train.json -beamsearch_size 2 -n_processes 71

In [ ]:
!cd MemSum/data_preprocessing; python get_high_rouge_episodes_mp.py -input_corpus_file_name ../../validation_data_parsed.json -output_corpus_file_name ../../custom_validation.json -beamsearch_size 2 -n_processes 71

In [ ]:
!cd MemSum/data_preprocessing; python get_high_rouge_episodes_mp.py -input_corpus_file_name ../../test_data_parsed.json -output_corpus_file_name ../../custom_test.json -beamsearch_size 2 -n_processes 71

In [ ]:
!cd MemSum/src; python train.py -training_corpus_file_name ../../custom_train.json -validation_corpus_file_name ../../custom_validation.json -model_folder ../../model/memsum-custom-data -log_folder ../../log/memsum-custom-data -vocabulary_file_name ../../model/word_embedding/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name ../../model/word_embedding/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 500 -num_of_epochs 50 -save_every 0 -validate_every 0 -n_device 1 -batch_size_per_device 4 -max_extracted_sentences_per_document 20 -moving_average_decay 0.999 -p_stop_thres 0.6

In [ ]:
!cd MemSum/src; python get_optimal_batch.py -log_file_name ../../log/memsum-custom-data/val.log

In [ ]:
from huggingface_hub import snapshot_download
## download the pretrained glove word embedding (200 dimension)
snapshot_download('nianlong/memsum-word-embedding', local_dir = "model/word_embedding" )

In [ ]:
from MemSum.src.summarizer import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

In [ ]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

In [ ]:
memsum_custom_data = MemSum(  "model/memsum-custom-data/model_batch_5600.pt", 
                  "model/word_embedding/vocabulary_200dim.pkl", 
                  gpu = 0 ,  max_doc_len = 500  )

In [ ]:
test_corpus_custom_data = [ json.loads(line) for line in open("custom_test.json")]

In [ ]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
        
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    
    return np.asarray(scores).mean(axis = 0)

In [ ]:
evaluate(memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal)


In [ ]:
document = test_corpus_custom_data[2]["text"]
extracted_summary = memsum_custom_data.extract( [ document ], 
                                   p_stop_thres = 0.6, 
                                   max_extracted_sentences_per_document = 50
                                  )[0]
print(document)
for element in document:
    if element in extracted_summary:
        print(element)